In [ ]:
import pathlib as pl
import sys
import datetime as dt

import pandas as pd
import geopandas as gpd
import r5py as r5
import pyrosm as pr
import geohexgrid as ghg
from loguru import logger

sys.path.append("../")

import r5py_isochrones as r5i

gpd.options.io_engine = "pyogrio"  # For faster geometric operations


# Setup

In [ ]:
DATA_DIR = pl.Path("../data") 
WGS84 = "epsg:4326"
NZTM = "epsg:2193"
%ls {DATA_DIR}


# Illustrate the functions 

I wrote two isochrone functions using r5py to help address [this Github issue](https://github.com/r5py/r5py/issues/311) .
One is based on a user-specified set of grid cells and
one based on the [concave hull isochrones of r5r](https://github.com/ipeaGIT/r5r/blob/master/r-package/R/isochrone.R).
I find the grid-style function more useful for my work (which often involves gridding up a study area for sampling),
because (1) i can set the resolution of my isochrones (via the grid), 
and (2) i needn't iterate to find a perfect convex hull ratio to use for accurate-looking results.

These examples use data from my location of Auckland, New Zealand, which i can confidently scrutinise.

In [ ]:
# Load Auckland OSM and GTFS data
%time akl_pbf_path = pr.get_data("Auckland", directory=DATA_DIR)
akl_gtfs_path = DATA_DIR / "auckland_gtfs_20230824.zip"

# Make the R5 transport network from these
%time transport_network = r5.TransportNetwork(akl_pbf_path, [akl_gtfs_path])
transport_modes = [
    r5.TransportMode.TRANSIT,
    r5.TransportMode.WALK,
]

In [ ]:
%%time

# Make a hexagon grid of circumradius 100 m covering the transport network
# and clipping to coastlines
study_area = (
    # Contains coastlines
    gpd.read_file(DATA_DIR / "auckland.gpkg")  
    .clip(
        gpd.GeoDataFrame(geometry=[transport_network.extent], crs=WGS84)
        .to_crs(NZTM)
    )
)
path = DATA_DIR / "auckland_grid_100m.gpkg"
if not path.exists():
    logger.info("Making grid")
    grid = ghg.make_grid_from_gdf(study_area, 100, clip=True)
    grid.to_file(path, driver="GPKG")
else:
    grid = gpd.read_file(path)

display(grid.head())
print("#grid cells =", grid.shape[0])

# Plot the study area and the some grid cells.
# The entire grid takes too long to plot
pd.concat([study_area, grid.iloc[55_000:56_000]]).explore(tiles="CartoDB positron")


In [ ]:
# Get some origin points, some of which might involve ferry rides

origins = gpd.read_file(DATA_DIR / "auckland_points.geojson").assign(id=lambda x: x.index).to_crs(NZTM)
display(origins)
display(origins.assign(geometry=lambda x: x.buffer(50)).explore(tiles="CartoDB positron"))


In [ ]:
%%time

# Takes ~15 seconds on my computer
isos = (
    r5i.isochrone_g(
        transport_network=transport_network, 
        transport_modes=transport_modes,
        origins=origins,
        departure=dt.datetime(2023, 8, 28, 8, 0, 0),
        time_bounds=[45],
        grid=grid.rename(columns={"cell_id": "id"}),
        departure_time_window=dt.timedelta(seconds=35*60),
        percentiles=[1, 25, 50],
    )
)

display(isos.head())
for ptile, group in isos.groupby("travel_time_percentile"):
    print(ptile)
    display(group.explore(column="from_id", categorical=True, cmap="viridis", tiles="CartoDB positron"))

In [ ]:
%%time

# Takes ~77 seconds on my computer
isos = (
    r5i.isochrone_ch(
        transport_network=transport_network, 
        transport_modes=transport_modes,
        origins=origins,
        departure=dt.datetime(2023, 8, 28, 8, 0, 0),
        time_bounds=[45],
        departure_time_window=dt.timedelta(seconds=35*60),
        percentiles=[1, 25, 50],
    )
)

display(isos.head())
for ptile, group in isos.groupby("travel_time_percentile"):
    print(ptile)
    display(group.explore(column="from_id", categorical=True, cmap="viridis", tiles="CartoDB positron"))